In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag
from datetime import date
import os

In [19]:
spark = SparkSession.builder.appName("Test connect to Postgresql") \
        .config('spark.jars.packages', 'org.postgresql:postgresql:42.7.3') \
        .getOrCreate()
        
povertyfact_df = spark.read.format("jdbc") \
    .option("url", f"{os.getenv('URL_BASE_LOCAL')}:{os.getenv('PORT')}/LdtbxhStage") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", 'public."stgPovertyStatusFact"') \
    .option("user", "postgres") \
    .option("password", "nhanbui") \
    .load()
    
member_df = spark.read.format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5434/hongheovna") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", 'public.family_member_info') \
    .option("user", "postgres") \
    .option("password", "nhanbui") \
    .load()
    
memberSurveyFact_df = spark.read.format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5434/LdtbxhStage") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", 'public."stgMemberSurveyFact"') \
    .option("user", "postgres") \
    .option("password", "nhanbui") \
    .load()
    
print("============================")
print(povertyfact_df.printSchema())
print(member_df.printSchema())
print(memberSurveyFact_df.printSchema())
print("============================")

Py4JJavaError: An error occurred while calling o162.load.
: org.postgresql.util.PSQLException: The connection attempt failed.
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:358)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:54)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:273)
	at org.postgresql.Driver.makeConnection(Driver.java:446)
	at org.postgresql.Driver.connect(Driver.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1(JdbcDialects.scala:160)
	at org.apache.spark.sql.jdbc.JdbcDialect.$anonfun$createConnectionFactory$1$adapted(JdbcDialects.scala:156)
	at org.apache.spark.sql.execution.d

In [26]:
def grade_difference(povertyfact_df):
    # create window for ordered dataframe
    # create previous-value columns by "lag" function
    windowSpec = Window.partitionBy("family_code").orderBy(["family_code", "year", "b1_grade", "b2_grade"])
    temp_b1 = lag("b1_grade").over(windowSpec).cast("integer")
    temp_b2 = lag("b2_grade").over(windowSpec).cast("integer")

    # Calculate the difference between current and previous b1_grade, b2_grade
    df = povertyfact_df.withColumn("b1_diff", col('b1_grade') - temp_b1)
    df = df.withColumn("b2_diff", col('b2_grade') - temp_b2)
    return df

In [27]:
def count_member(povertyfact_df):
    # create a dataframe for number of member each family_id
    count_df = member_df.groupBy("family_id").count()

    # join povertyfact to count_df and assign "count" value to "member_num" -> drop "count" column at the end
    joined_df = povertyfact_df.join(count_df, on="family_id", how="left")
    updated_df = joined_df.withColumn("member_num", joined_df["count"])
    final_df = updated_df.drop("count")
    return final_df

In [14]:
def find_age_member(memberSurveyFact_df):
    windowSpec = Window.partitionBy("member_id")
    final_df = memberSurveyFact_df.withColumn('age', date.today().year - col('year_of_birth'))
    
    return final_df

In [26]:
f"{os.getenv('URL_BASE_LOCAL')}:{os.getenv('PORT')}/LdtbxhStage"

'jdbc:postgresql://localhost:5434:5434/LdtbxhStage'

In [30]:
import json

with open("./airflow/config.json", "r") as file:
        config = json.load(file)
    
spark = SparkSession.builder.appName("Test connect to Postgresql") \
        .config('spark.jars.packages', 'org.postgresql:postgresql:42.7.3') \
        .getOrCreate()
    
memberSurveyFact_df = spark.read.format("jdbc") \
    .option("url", f"{config['URL_BASE_DOCKER']}:{config['PORT']}/LdtbxhStage") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", 'public."stgMemberSurveyFact"') \
    .option("user", "postgres") \
    .option("password", "nhanbui") \
    .load()

finalfact_df = find_age_member(memberSurveyFact_df)

print("====================")
print(finalfact_df.show(5))
print("====================")

finalfact_df.write.format("jdbc") \
    .option("url", f"{config['URL_BASE_DOCKER']}:{config['PORT']}/LdtbxhStage") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", 'public."stgMemberSurveyFact"') \
    .option("user", "postgres") \
    .option("password", "nhanbui") \
    .mode("overwrite") \
    .save()

print("====================")
print(finalfact_df.show(5))
print("====================")

FileNotFoundError: [Errno 2] No such file or directory: './airflow/config.json'

In [29]:
run_gold()

+--------------------+----+-------------+-------------+-----------+----------+--------------------+--------------------+--------------------+----------+-------+--------+--------+-------+-------+------------+
|           family_id|year|province_name|district_name|family_code|owner_name|        hard_reasons|        get_policies|       need_policies|member_num|a_grade|b1_grade|b2_grade|b1_diff|b2_diff|final_result|
+--------------------+----+-------------+-------------+-----------+----------+--------------------+--------------------+--------------------+----------+-------+--------+--------+-------+-------+------------+
|966a5c75-d6f5-48c...|2022|     Đắk Nông|       Cư Jút|       7009|      NULL|[không có đất sản...|[hỗ trợ y tế, hỗ ...|[hỗ trợ nước sạch...|      NULL|   true|      45|      65|   NULL|   NULL|    hộ nghèo|
|f4773df9-32bc-417...|2023|     Đắk Nông|       Cư Jút|       7003|  Độ A Đạt|[không có đất sản...|[hỗ trợ y tế, hỗ ...|[hỗ trợ nước sạch...|         1|   true|      30